# Web Crawling

In [3]:
import FinanceDataReader as fdr
import pandas as pd
from bs4 import BeautifulSoup
import datetime as dt
from datetime import datetime,timedelta
from urllib.request import urlopen
import sqlalchemy 
import pymysql
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

# MoneyTrend

In [ ]:
def get_money_trend():
    
    url = 'http://finance.naver.com/sise/sise_deposit.nhn?&page='

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    last = source.find('td',class_='pgRR').find('a')['href']
    last = last.split('&')[1]
    last = last.split('=')[1]
    last = int(last)

    # 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
    path = 'd:\\moneytrend.xlsx'   
    
    # 날짜를 받을 리스트
    date_list = []

    # 값을 받을 사전
    dictionary = {'고객예탁금': [],'신용잔고': [],'주식형 펀드': [],'혼합형 펀드': [],'채권형 펀드': []}

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['고객예탁금','신용잔고','주식형 펀드','혼합형 펀드','채권형 펀드']


    # count mask
    mask = [1,3,5,7,9]
    
    for i in range(1,last+1):
        
        source = urlopen(url+ str(i)).read()
        source = BeautifulSoup(source,'lxml')

        #tbody = source.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})
        #trs = tbody.find_all('tr')

        body = source.find('body')
        trs = body.find_all('tr')

        for tr in trs:
            tds = tr.find_all('td',{'class':['date','rate_down','rate_up']})
            count = 0
    
            for td in tds:
                if count == 0:
                    date_ = td.text.strip().replace('.','-')
                    date_list.append(date_)
                        
                      
                elif count in mask:
                    temp = int((count-1)/2)
                    dictionary[name_list[temp]].append(td.text.strip().replace(',',''))
        
                count += 1
            if len(dictionary['고객예탁금']) != len(dictionary['채권형 펀드']):
                print(str(i)+ '번째 페이지에서 누락된 값 발생')
                print('누락된 데이터를 제거합니다')
                    
                date_list.pop(-1)
                dictionary['고객예탁금'].pop(-1)
                dictionary['신용잔고'].pop(-1)
                dictionary['주식형 펀드'].pop(-1)
                dictionary['혼합형 펀드'].pop(-1)
                
    # 개별 list 요소 갯수 파악 
    #print(len(date_list))
    #print(len(dictionary['고객예탁금']))
    #print(len(dictionary['신용잔고']))
    #print(len(dictionary['주식형 펀드']))
    #print(len(dictionary['혼합형 펀드']))
    #print(len(dictionary['채권형 펀드']))
    print(str(i) + '번째 페이지 크롤링 완료')
    df = pd.DataFrame(dictionary,index = date_list)
    df = df.sort_index()
    df.to_excel(path, encoding='utf-8')
    print(df)


get_money_trend()



In [5]:
def get_money_trend_date():
    url = 'http://finance.naver.com/sise/sise_deposit.nhn?&page='

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    last = source.find('td',class_='pgRR').find('a')['href']
    last = last.split('&')[1]
    last = last.split('=')[1]
    last = int(last)

    # 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
    path = 'd:\\money_trend.xlsx'

    
    until_date = input("날짜를 입력하세요 sample: '2019-01-10': ")
    
    year = until_date.split('-')[0]
    mm = until_date.split('-')[1]
    dd = until_date.split('-')[2]
    year=year[2:]
    until_date = year+'-'+mm+'-'+dd
    
    #df = DataFrame(columns = ['고객예탁금', '신용잔고','주식형 펀드','혼합형 펀드','채권형 펀드'])

    # 날짜를 받을 리스트
    date_list = []

    
    # 값을 받을 사전
    dictionary = {'고객예탁금': [],
                  '신용잔고': [],
                  '주식형 펀드': [],
                  '혼합형 펀드': [],
                  '채권형 펀드': []
                  }

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['고객예탁금','신용잔고','주식형 펀드','혼합형 펀드','채권형 펀드']


    # count mask
    mask = [1,3,5,7,9]
    
    for i in range(1,last+1):
        
        source = urlopen(url+ str(i)).read()
        source = BeautifulSoup(source,'lxml')

        #tbody = source.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})
        #trs = tbody.find_all('tr')

        body = source.find('body')
        trs = body.find_all('tr')

        for tr in trs:
            tds = tr.find_all('td',{'class':['date','rate_down','rate_up']})
            count = 0
    
            for td in tds:
                if count == 0:
                    date_ = td.text.strip().replace('.','-')
                    if date_ <=  until_date :
                    #if date_ <=  '19-03-05' :
                        df = pd.DataFrame(dictionary,index = date_list)
                        df = df.sort_index()
                        df.to_excel(path, encoding='utf-8')
                        return df
                    date_list.append(date_)
                    
                elif count in mask:
                    temp = int((count-1)/2)
                    dictionary[name_list[temp]].append(td.text.strip().replace(',',''))
                
       
                count += 1

get_money_trend_date()

날짜를 입력하세요 sample: '2019-01-10': 2019-03-08


,고객예탁금,신용잔고,주식형 펀드,혼합형 펀드,채권형 펀드
19-03-11,"249,915","103,175","861,947","275,543","1,076,049"
19-03-12,"241,209","103,111","864,535","275,569","1,072,923"
19-03-13,"248,093","103,310","861,124","275,556","1,073,121"


# KPI200

In [ ]:
def get_kpi200():
    
    url = 'https://finance.naver.com/sise/sise_index_day.nhn?code=KPI200&page='

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    last = source.find('td',class_='pgRR').find('a')['href']
    last = last.split('page')[1]
    last = last.split('=')[1]
    last = int(last)
    print(last)

    # 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
    path = 'd:\\kpi200.xlsx'
    
    # 날짜를 받을 리스트
    date_list = []

    # 값을 받을 사전
    dictionary = {'KPI200': [],'거래량': []}

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['KPI200','거래량']


    # count mask
    mask = [1,3]
    
    for i in range(1,last+1):
        
        source = urlopen(url+ str(i)).read()
        source = BeautifulSoup(source,'lxml')

        #tbody = source.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})
        #trs = tbody.find_all('tr')

        body = source.find('body')
        trs = body.find_all('tr')

        for tr in trs:
            tds = tr.find_all('td',{'class':['date','number_1']})
            count = 0
    
            for td in tds:
                if count == 0:
                    date_ = td.text.strip().replace('.','-')
                    date_list.append(date_)
                        
                      
                elif count in mask:
                    temp = int(count/3)
                    dictionary[name_list[temp]].append(td.text.strip().replace(',',''))
        
                count += 1
            if len(date_list) != len(dictionary['KPI200']):
                print(str(i)+ '번째 페이지에서 누락된 값 발생')
                print('누락된 데이터를 제거합니다')
                    
                date_list.pop(-1)
                #dictionary['KPI200'].pop(-1)
                dictionary['거래량'].pop(-1)
                
    # 개별 list 요소 갯수 파악 
    #print(len(date_list))
    #print(len(dictionary['개인']))
    #print(len(dictionary['외국인']))
    #print(len(dictionary['기관']))

    print(str(i) + '번째 페이지 크롤링 완료')
    df = pd.DataFrame(dictionary,index = date_list)
    df = df.sort_index()
    df.to_excel(path, encoding='utf-8')
    print(df)

get_kpi200()

In [ ]:
def get_kpi200_date():
    
    url = 'https://finance.naver.com/sise/sise_index_day.nhn?code=KPI200&page='

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    last = source.find('td',class_='pgRR').find('a')['href']
    last = last.split('page')[1]
    last = last.split('=')[1]
    last = int(last)
    print(last)

    # 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
    path = 'd:\\kpi200.xlsx'

    until_date = input("날짜를 입력하세요 sample: '2019-01-10': ")
    
    year = until_date.split('-')[0]
    mm = until_date.split('-')[1]
    dd = until_date.split('-')[2]
    #year=year[2:]
    until_date = year+'-'+mm+'-'+dd
    
    # 날짜를 받을 리스트
    date_list = []

    # 값을 받을 사전
    dictionary = {'KPI200': [],'거래량': []}

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['KPI200','거래량']


    # count mask
    mask = [1,3]
    
    for i in range(1,last+1):
        
        source = urlopen(url+ str(i)).read()
        source = BeautifulSoup(source,'lxml')

        #tbody = source.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})
        #trs = tbody.find_all('tr')

        body = source.find('body')
        trs = body.find_all('tr')

        for tr in trs:
            tds = tr.find_all('td',{'class':['date','number_1']})
            count = 0
    
            for td in tds:
                if count == 0:
                    date_ = td.text.strip().replace('.','-')
                    if date_ <=  until_date :
                    #if date_ <=  '19-03-05' :
                        df = pd.DataFrame(dictionary,index = date_list)
                        df = df.sort_index()
                        df.to_excel(path, encoding='utf-8')
                        return df   
                    date_list.append(date_)
                    #print(date_list)
                elif count in mask:
                    temp = int(count/3)
                    dictionary[name_list[temp]].append(td.text.strip().replace(',',''))
                    #print(dictionary[name_list[temp]])
                count += 1
                
get_kpi200_date()

# Investor Trend

In [ ]:
def get_investor_trend_date():
    
    url = 'http://finance.naver.com/sise/investorDealTrendDay.nhn?bizdate=2020601&sosok=&page='

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    last = source.find('td',class_='pgRR').find('a')['href']
    last = last.split('page')[1]
    last = last.split('=')[1]
    last = int(last)
    print(last)

    # 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
    path = 'd:\\investortrend.xlsx'

    until_date = input("날짜를 입력하세요 sample: '2019-01-10': ")
    
    year = until_date.split('-')[0]
    mm = until_date.split('-')[1]
    dd = until_date.split('-')[2]
    year=year[2:]
    until_date = year+'-'+mm+'-'+dd
    
    # 날짜를 받을 리스트
    date_list = []

    # 값을 받을 사전
    dictionary = {'개인': [],'외국인': [],'기관': []}

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['개인','외국인','기관']


    # count mask
    mask = [1,2,3]
    
    for i in range(1,last+1):
        
        source = urlopen(url+ str(i)).read()
        source = BeautifulSoup(source,'lxml')

        #tbody = source.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})
        #trs = tbody.find_all('tr')

        body = source.find('body')
        trs = body.find_all('tr')

        for tr in trs:
            tds = tr.find_all('td',{'class':['date2','rate_down3','rate_up3']})
            count = 0
    
            for td in tds:
                if count == 0:
                    date_ = td.text.strip().replace('.','-')
                    if date_ <=  until_date :
                        df = pd.DataFrame(dictionary,index = date_list)
                        df = df.sort_index()
                        df.to_excel(path, encoding='utf-8')
                        return df   
                    date_list.append(date_)
                    #print(date_list)
                elif count in mask:
                    temp = int(count-1)
                    dictionary[name_list[temp]].append(td.text.strip().replace(',',''))
                    
                count += 1
get_investor_trend()

# Program Trend

In [ ]:
def get_program_trend():
    
    url = 'https://finance.naver.com/sise/programDealTrendDay.nhn?bizdate=20190315&sosok=&page='

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    last = source.find('td',class_='pgRR').find('a')['href']
    last = last.split('page')[1]
    last = last.split('=')[1]
    last = int(last)
    print(last)

    # 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
    path = 'd:\\programtrend.xlsx'
    
    # 날짜를 받을 리스트
    date_list = []

    # 값을 받을 사전
    dictionary = {'차익': [],'비차익': [],'전체': []}

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['차익','비차익','전체']


    # count mask
    mask = [3,6,9]
    
    for i in range(1,last+1):
        
        source = urlopen(url+ str(i)).read()
        source = BeautifulSoup(source,'lxml')

        #tbody = source.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})
        #trs = tbody.find_all('tr')

        body = source.find('body')
        trs = body.find_all('tr')

        for tr in trs:
            tds = tr.find_all('td',{'class':['date','rate_down','rate_up']})
            count = 0
    
            for td in tds:
                if count == 0:
                    date_ = td.text.strip().replace('.','-')
                    date_list.append(date_)
                        
                      
                elif count in mask:
                    temp = int((count/3)-1)
                    dictionary[name_list[temp]].append(td.text.strip().replace(',',''))
        
                count += 1
            if len(date_list) != len(dictionary['전체']):
                print(str(i)+ '번째 페이지에서 누락된 값 발생')
                print('누락된 데이터를 제거합니다')
                    
                date_list.pop(-1)
                dictionary['차익'].pop(-1)
                dictionary['비차익'].pop(-1)
                #dictionary['전체'].pop(-1)
                
    # 개별 list 요소 갯수 파악 
    print(len(date_list))
    print(len(dictionary['차익']))
    print(len(dictionary['비차익']))
    print(len(dictionary['전체']))

    print(str(i) + '번째 페이지 크롤링 완료')
    df = pd.DataFrame(dictionary,index = date_list)
    df = df.sort_index()
    df.to_excel(path, encoding='utf-8')
    print(df)

get_program_trend()

In [20]:
def get_program_trend_date():
    
    url = 'https://finance.naver.com/sise/programDealTrendDay.nhn?bizdate=20190315&sosok=&page='

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    last = source.find('td',class_='pgRR').find('a')['href']
    last = last.split('page')[1]
    last = last.split('=')[1]
    last = int(last)
    print(last)

    # 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
    path = 'd:\\investortrend.xlsx'

    until_date = input("날짜를 입력하세요 sample: '2019-01-10': ")
    
    year = until_date.split('-')[0]
    mm = until_date.split('-')[1]
    dd = until_date.split('-')[2]
    year=year[2:]
    until_date = year+'-'+mm+'-'+dd
    
    # 날짜를 받을 리스트
    date_list = []

    # 값을 받을 사전
    dictionary = {'차익': [],'비차익': [],'전체': []}

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['차익','비차익','전체']


    # count mask
    mask = [1,2,3]
    
    for i in range(1,last+1):
        
        source = urlopen(url+ str(i)).read()
        source = BeautifulSoup(source,'lxml')

        #tbody = source.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})
        #trs = tbody.find_all('tr')

        body = source.find('body')
        trs = body.find_all('tr')

        for tr in trs:
            tds = tr.find_all('td',{'class':['date','rate_down','rate_up']})
            count = 0
    
            for td in tds:
                if count == 0:
                    date_ = td.text.strip().replace('.','-')
                    if date_ <=  until_date :
                        df = pd.DataFrame(dictionary,index = date_list)
                        df = df.sort_index()
                        df.to_excel(path, encoding='utf-8')
                        return df   
                    date_list.append(date_)
                    #print(date_list)
                elif count in mask:
                    temp = int(count-1)
                    dictionary[name_list[temp]].append(td.text.strip().replace(',',''))
                    
                count += 1
get_program_trend_date()

352
날짜를 입력하세요 sample: '2019-01-10': 2019-03-01


,차익,비차익,전체
19-03-04,1168,1453,-285
19-03-05,1797,2161,-364
19-03-06,1289,1807,-518
19-03-07,1450,1698,-248
19-03-08,1345,1560,-215
19-03-11,1220,1334,-114
19-03-12,1188,1327,-139
19-03-13,1218,1513,-294
19-03-14,2228,3020,-792
19-03-15,2176,1756,420


# Test

In [5]:
def get_program_trend():
    
    url = 'https://finance.naver.com/sise/programDealTrendDay.nhn?bizdate=20190315&sosok=&page=1'

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    last = source.find('td',class_='pgRR').find('a')['href']
    last = last.split('page')[1]
    last = last.split('=')[1]
    last = int(last)
    print(last)
get_program_trend()



352


In [1]:
import requests

In [8]:
import requests
import json
import pandas as pd

url = 'http://finance.daum.net/api/trend/arounds?page=1&perPage=20&fieldName=changeRate&order=desc&'

headers = {
            'Accept': 'application/json, text/javascript, */*; q=0.01',
            'Accept-Encoding': 'gzip, deflate',
            'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
            'Connection': 'keep-alive',
            'Cookie': 'webid=d46440de38044bf09793bcb7b72f6e44; HM_CU=53tOIE3pJAl; PROF=0603012032024076024152UiQPJk7X-6w0mlxoempuua9ZV2cHpAw.yYZCWRveAG66KuZP3z7wEl2bxriNh5SdkQ00LYYSA9A1_cGNLCyhCzrwOpjlf.dyaYOF73J4wVWlUV-s4YfMaxSVtlykILG9ANzoDGNdZmMA6ow0fIauQlo8OTLLnHHY.bHDTw00pCsuaN7PcHxD-1-COJRcrCRKdtGkJTxsMPDn.AARqchjj_-wex3iZx3wLHSw2zOJV.v6jIW4KBMTlbLOnujI-eVINNr9BbQ_KeOzYuEMT_ef42fOXQqNBos4gXyk4bhTSZyZ2Y1KFae9xnaQoQWr5w00; SLEVEL=1; ssab=; AGEN=_mu9pzAMt5sQLPv-ldWVC_IAFijAIG9qARn2QDRvByo; TS=1553412011; HTS=CiiEXdlRFxG7ojGJ9WgV1w00; LSID=f6127be3-d5e5-4032-8c37-628ae599bd181553412011772; _ga=GA1.2.425358004.1553636955; _gid=GA1.2.296667810.1553636955; webid_sync=1553723295574; _gat_gtag_UA_128578811_1=1; TIARA=33ct94Z7b1leKyZ3bC-NI856RU7.OHSANrbu4xUyWnUvsKKAGyYHKdKtFobGx.DrFpAAYU4lBaLcWieRt6trQmWZ_AYwcswz',
            'Host': 'finance.daum.net',
            'If-None-Match': 'W/"486e9e8ca2eb9ee4cfa1c386a42117f3"',
            'Referer': 'http://finance.daum.net/domestic/arounds',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'
            }
headers['Referer'] = 'http://finance.daum.net/api/trend/arounds?page=1&perPage=20&fieldName=changeRate&order=desc&'
r = requests.get(url, headers = headers)

data = json.loads(r.text)
df = pd.DataFrame(data['data'])

return df

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [4]:
import requests
import json
import pandas as pd

def get_price(code, n):
    # DATA를 불러오는 부분 입니다.
    url = 'http://finance.daum.net/api/charts/A%s/days?limit=%d&adjusted=true'%(code, n)
    headers = {
                'Accept': 'application/json, text/plain, */*',
                'Accept-Encoding': 'gzip, deflate',
                'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
                'Connection': 'keep-alive',
                'Cookie': 'GS_font_Name_no=0; GS_font_size=16; _ga=GA1.3.937989519.1493034297; webid=bb619e03ecbf4672b8d38a3fcedc3f8c; _ga=GA1.2.937989519.1493034297; _gid=GA1.2.215330840.1541556419; KAKAO_STOCK_RECENT=[%22A069500%22]; recentMenus=[{%22destination%22:%22chart%22%2C%22title%22:%22%EC%B0%A8%ED%8A%B8%22}%2C{%22destination%22:%22current%22%2C%22title%22:%22%ED%98%84%EC%9E%AC%EA%B0%80%22}]; TIARA=C-Tax5zAJ3L1CwQFDxYNxe-9yt4xuvAcw3IjfDg6hlCbJ_KXLZZhwEPhrMuSc5Rv1oty5obaYZzBQS5Du9ne5x7XZds-vHVF; webid_sync=1541565778037; _gat_gtag_UA_128578811_1=1; _dfs=VFlXMkVwUGJENlVvc1B3V2NaV1pFdHhpNTVZdnRZTWFZQWZwTzBPYWRxMFNVL3VrODRLY1VlbXI0dHhBZlJzcE03SS9Vblh0U2p2L2V2b3hQbU5mNlE9PS0tcGI2aXQrZ21qY0hFbzJ0S1hkaEhrZz09--6eba3111e6ac36d893bbc58439d2a3e0304c7cf3',
                'Host': 'finance.daum.net',
                'If-None-Match': 'W/"23501689faaaf24452ece4a039a904fd"',
                'Referer': 'http://finance.daum.net/quotes/A069500',
                'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'
                }
    headers['Referer'] = 'http://finance.daum.net/quotes/A%s'%code
    r = requests.get(url, headers = headers)
    
    # DATA를 보기 좋게 편집하는 부분 입니다.
    data = json.loads(r.text)
    df = pd.DataFrame(data['data'])
    df.index = pd.to_datetime(df['candleTime'])
    
    return df

In [5]:
get_price('000020', 20)

,candleAccTradePrice,candleAccTradeVolume,candleTime,change,changePrice,changeRate,date,highPrice,lowPrice,openingPrice,symbolCode,timestamp,tradePrice,tradeTime
candleTime,,,,,,,,,,,,,,
2019-02-28,1262292450,127433,2019-02-28 00:00:00.0,FALL,130.0,0.013092,2019-02-28,9980.0,9720.0,9930.0,A000020,1551345978459,9800.0,153025
2019-03-04,858235130,86244,2019-03-04 00:00:00.0,RISE,200.0,0.020408,2019-03-04,10050.0,9800.0,9810.0,A000020,1551691539233,10000.0,153021
2019-03-05,655114470,65110,2019-03-05 00:00:00.0,RISE,100.0,0.010000,2019-03-05,10150.0,9970.0,10050.0,A000020,1551777979504,10100.0,153010
2019-03-06,484114800,47791,2019-03-06 00:00:00.0,EVEN,0.0,0.000000,2019-03-06,10250.0,10050.0,10150.0,A000020,1551864371128,10100.0,153030
2019-03-07,658441180,65924,2019-03-07 00:00:00.0,FALL,110.0,0.010891,2019-03-07,10100.0,9910.0,10050.0,A000020,1551950773359,9990.0,153010
2019-03-08,343866720,34591,2019-03-08 00:00:00.0,FALL,60.0,0.006006,2019-03-08,10000.0,9860.0,9990.0,A000020,1552037163192,9930.0,153005
2019-03-11,877700660,89651,2019-03-11 00:00:00.0,FALL,80.0,0.008056,2019-03-11,9960.0,9700.0,9930.0,A000020,1552296378581,9850.0,153024
2019-03-12,502122480,50658,2019-03-12 00:00:00.0,RISE,80.0,0.008122,2019-03-12,9960.0,9870.0,9900.0,A000020,1552382772168,9930.0,153030
2019-03-13,748344140,74778,2019-03-13 00:00:00.0,RISE,120.0,0.012085,2019-03-13,10150.0,9930.0,9950.0,A000020,1552469150813,10050.0,153022
